In [2]:
from collections import defaultdict

current = {'green': 12, 'bule': 3}
increments = [
    ('red', 5),
    ('blue', 17),
    ('orange', 9)
]

def log_missing():
    print("key added")
    return 0

result = defaultdict(log_missing, current)
print("Before:", dict(result))
for key, amount in increments:
    result[key] += amount
print("After:", dict(result))

('Before:', {'green': 12, 'bule': 3})
key added
key added
key added
('After:', {'blue': 17, 'orange': 9, 'green': 12, 'bule': 3, 'red': 5})


In [6]:
from collections import defaultdict


current = {'green': 12, 'bule': 3}
increments = [
    ('red', 5),
    ('blue', 17),
    ('orange', 9)
]


def increment_with_report(current, increments):
    added_count = 0
    
    def missing():
        nonlocal added_count
        added_count += 1
        return 0
    
    result = defaultdict(missing, current)
    for key, amount in increments:
        result[key] += amount
        
    return result, added_count

result, count = increment_with_report(current, increments)
assert count == 3


In [9]:
from collections import defaultdict


current = {'green': 12, 'bule': 3}
increments = [
    ('red', 5),
    ('blue', 17),
    ('orange', 9)
]


class CountMissing(object):
    def __init__(self):
        self.count = 0
        
    def missing(self):
        self.count += 1
        return 0
    
count = CountMissing()
result = defaultdict(count.missing, current)
for key, amount in increments:
    result[key] += amount
print(dict(result))
print(count.count)

{'blue': 17, 'orange': 9, 'red': 5, 'bule': 3, 'green': 12}
3


In [13]:
from collections import defaultdict


current = {'green': 12, 'bule': 3}
increments = [
    ('red', 5),
    ('blue', 17),
    ('orange', 9)
]


class BetterCountMissing(object):
    def __init__(self):
        self.count = 1
        
    def __call__(self):
        self.count += 1
        return 0
    
count = BetterCountMissing()
print(count())
assert callable(count)

result = defaultdict(count, current)
for key, amount in increments:
    result[key] += amount

print(dict(result))


0
{'blue': 17, 'orange': 9, 'red': 5, 'bule': 3, 'green': 12}


In [2]:
# cofing=utf-8
import os
from tempfile import TemporaryDirectory
import tempfile
from threading import Thread


class InputData(object):
    def read(self):
        raise NotImplementedError


class PathInputData(object):
    def __init__(self, path):
        super().__init__()
        self.path = path

    def read(self):
        return open(self.path, encoding="utf8").read()


class Worker(object):
    def __init__(self, input_data):
        self.input_data = input_data
        self.result = None

    def map(self):
        raise NotImplementedError

    def reduce(self):
        raise NotImplementedError


class LineCountWorker(Worker):
    def map(self):
        data = self.input_data.read()
        self.result = data.count("\n")
        print(self.result)

    def reduce(self, other):
        self.result += other.result


def generate_inputs(data_dir):
    for name in os.listdir(data_dir):
        if name.endswith("txt"):
            # print(os.path.join(data_dir, name))
            yield PathInputData(os.path.join(data_dir, name))


def creata_workers(input_list):
    workers = []
    for input_data in input_list:
        workers.append(LineCountWorker(input_data))
    return workers


def execute(workers):
    threads = [Thread(target=w.map) for w in workers]
    for thread in threads:
        thread.start()
    for thread in threads:
        thread.join()

    first, rest = workers[0], workers[1:]
    for worker in rest:
        first.reduce(worker)
    return first.result


def mapreduce(data_dir):
    input_list = generate_inputs(data_dir)
    workers = creata_workers(input_list)
    return execute(workers)


with TemporaryDirectory() as tmpdir:
    # tmpdir = r"C:\Users\zxc\AppData\Local\Temp\tPyUxDiMxXjRiNmX"
    tmpdir = r"F:\Coding.net\test\jieba_wordcloud"
    result = mapreduce(tmpdir)

print("There are", result, "lines")

58
18939

There are 1960 lines


In [8]:
import os
from threading import Thread


class GenericInputData(object):
    def __init__(self, path):
        super().__init__()
        self.path = path

    def read(self):
        raise NotImplementedError

    @classmethod
    def generic_input(cls, config):
        raise NotImplementedError


class PathInputData(GenericInputData):
    def read(self):
        return open(self.path, encoding="utf8").read()

    @classmethod
    def generic_input(cls, config):
        for name in os.listdir(config):
            if name.endswith(".txt"):
                yield cls(os.path.join(config, name))


class GenericWorker(object):
    def __init__(self, input_data):
        self.input_data = input_data
        self.result = None

    def map(self):
        raise NotImplementedError

    def reduce(self):
        raise NotImplementedError

    @classmethod
    def create_workers(cls, input_class, config):
        workers = []
        for input_data in input_class.generic_input(config):
            workers.append(cls(input_data))
        return workers


class LineCountWorker(GenericWorker):
    def map(self):
        data = self.input_data.read()
        self.result = data.count("\n")
        print(self.result)

    def reduce(self, other):
        self.result += other.result


def execute(workers):
    threads = [Thread(target=w.map) for w in workers]
    for thread in threads:
        thread.start()
    for thread in threads:
        thread.join()

    first, rest = workers[0], workers[1:]
    for worker in rest:
        first.reduce(worker)
    return first.result


def mapreduce(worker_class, input_class, config):
    workers = worker_class.create_workers(input_class, config)
    return execute(workers)


tmpdir = r"F:\Coding.net\test\jieba_wordcloud"
result = mapreduce(LineCountWorker, PathInputData, tmpdir)

print("There are", result, "lines")


58
9
1893
There are 1960 lines


In [1]:
import os
from tempfile import TemporaryDirectory


with TemporaryDirectory() as tmpdir:
    print(tmpdir)
    for name in os.listdir(os.path.curdir):
        print(name)
    else:
        print(123)

C:\Users\zxc\AppData\Local\Temp\tmp2kejubi4
.git
.gitignore
.ipynb_checkpoints
cafe.txt
Design Patterns.ipynb
Effective Python.ipynb
Fluent Python.ipynb
Numpy .ipynb
Practice——Python2.ipynb
Practice——Python3.ipynb
PythonSheets.ipynb
README.md
sqlalchemy.ipynb
test.ipynb
Test——Python2.ipynb
WebDevelop.ipynb
123


In [14]:
from pprint import pprint

class MyBaseClass(object):
    def __init__(self, value):
        self.value = value

class TimesTwo(object):
    def __init__(self, value):
        self.value *= 2
    
class PlusFive(object):
    def __init__(self, value):
        self.value += 5
        
class FirstWay(MyBaseClass, TimesTwo, PlusFive):
    def __init__(self, value):
        MyBaseClass.__init__(self, value)
        TimesTwo.__init__(self, value)
        PlusFive.__init__(self, value)
class SecondWay(MyBaseClass, PlusFive, TimesTwo):
    def __init__(self, value):
        MyBaseClass.__init__(self, value)
        TimesTwo.__init__(self, value)
        PlusFive.__init__(self, value)
        
        
class TimesFive(MyBaseClass):
    def __init__(self, value):
        MyBaseClass.__init__(self, value)
        self.value *= 5
        
class TimesFiveCorrect(MyBaseClass):
    def __init__(self, value):
        super(TimesFiveCorrect, self).__init__(value)
        self.value *= 5
        
class PlusTwo(MyBaseClass):
    def __init__(self, value):
        MyBaseClass.__init__(self, value)
        self.value += 2

class PlusTwoCorrect(MyBaseClass):
    def __init__(self, value):
#         super(PlusTwoCorrect, self).__init__(value)
        super(__class__, self).__init__(value)
        self.value += 2
        
class ThisWay(TimesFive, PlusTwo):
    def __init__(self, value):
        TimesFive.__init__(self, value)
        PlusTwo.__init__(self, value)

class SuperWay(TimesFiveCorrect, PlusTwoCorrect):
    def __init__(self, value):
#         super(SuperWay, self).__init__(value)
        super().__init__(value)
        
fw = FirstWay(2)
print(fw.value)
sw = SecondWay(2)
print(sw.value)
tw = ThisWay(5)
print(tw.value)
tw = ThisWay(4)

print(tw.value)
pprint(ThisWay.mro())
# 因为是菱形继承，会将value值初始化两次

sw = SuperWay(5)
print(sw.value)
pprint(SuperWay.mro())
# 5*(5+2) = 35

9
9
7
6
[<class '__main__.ThisWay'>,
 <class '__main__.TimesFive'>,
 <class '__main__.PlusTwo'>,
 <class '__main__.MyBaseClass'>,
 <class 'object'>]
35
[<class '__main__.SuperWay'>,
 <class '__main__.TimesFiveCorrect'>,
 <class '__main__.PlusTwoCorrect'>,
 <class '__main__.MyBaseClass'>,
 <class 'object'>]


In [17]:
class MyOtherObject(object):
    def __init__(self):
         self.__private_value = 5
            
    @classmethod
    def get_private_from_instance(cls, instance):
        return instance.__private_value
    
a = MyOtherObject()
print(a.__dict__)
print(MyOtherObject.get_private_from_instance(a))
assert MyOtherObject.get_private_from_instance(a) == 5

{'_MyOtherObject__private_value': 5}
5


In [29]:
class MyClass:
    def __init__(self, value):
        self.__value = value
        
    def get_value(self):
        return str(self.__value)
    
class MySubClass(MyClass):
    def get_value(self):
        return self._MyClass__value

mc = MyClass(10)    
print(mc.get_value())
foo = MySubClass(15)
print(foo.get_value())

10
15


In [34]:
class FrequencyList(list):
    def __init__(self, members):
        super().__init__(members)
    def frequency(self):
        counts = {}
        for item in self:
            counts.setdefault(item, 0)
            counts[item] += 1
        return counts

a = 'a'
b = 'b'
c = 'c'
members = [a, a, a, b, b, c]
print(FrequencyList(members).frequency())


{'b': 2, 'a': 3, 'c': 1}


In [2]:
class TestGetattr(object):
    def __init__(self):
        self._exists = 5

    def __getattr__(self, name):
        value = "Value for {}".format(name)
        setattr(self, name, value)
        return value

tg = TestGetattr()
print("Before:", tg.__dict__)
tg.foo
print("After:", tg.__dict__)


class TestGetattrSub(TestGetattr):
    # def __getattr__(self, name):
    #     print("Called __getattr__ {}".format(name))
    #     return super(TestGetattrSub, self).__getattr__(name)

    def __getattribute__(self, name):
        print("Called __getattribute__ {}".format(name))
        try:
            return super(TestGetattrSub, self).__getattribute__(name)
        except AttributeError:
            print("Not have this value")
            value = "Value for {}".format(name)
            setattr(self, name, value)
            return value


class TestGetattrClass(object):
    def __getattr__(self, name):
        print("Called __getattr__ {}".format(name))
        return super(TestGetattrClass, self).__getattr__(name)

    def __getattribute__(self, name):
        print("Called __getattribute__ {}".format(name))
        try:
            return super(TestGetattrClass, self).__getattribute__(name)
        except AttributeError:
            print("Not have this value")
            value = "Value for {}".format(name)
            setattr(self, name, value)
            return value


class TestSetattrClass():
    def __setattr__(self, name, value):
        print("Called __setattr__ {}{}".format(name, value))
        super(TestSetattrClass, self).__setattr__(name, value)


tgs = TestGetattrSub()
print("Before:", tgs.__dict__)
print("foo:", tgs.foo)  # 第一次访问是不存在，然后会调用__getattr__
print("foo:", tgs.foo)  # 第二次访问是，因为已经存在于__dict__中了
print("foo:", tgs.foo)
print("After:", tgs.__dict__)
print("--"*40)
tgc = TestGetattrClass()
print("Before:", tgc.__dict__)
print("foo:", tgc.foo)
print("foo:", tgc.foo)
print("After:", tgc.__dict__)
print("--"*40)
tsc = TestSetattrClass()
print("Before:", tsc.__dict__)
tsc.data = 5
print("After:", tsc.__dict__)
tsc.data = 10
print("After:", tsc.__dict__)


Before: {'_exists': 5}
After: {'foo': 'Value for foo', '_exists': 5}
Called __getattribute__ __dict__
Before: {'_exists': 5}
Called __getattribute__ foo
Not have this value
foo: Value for foo
Called __getattribute__ foo
foo: Value for foo
Called __getattribute__ foo
foo: Value for foo
Called __getattribute__ __dict__
After: {'foo': 'Value for foo', '_exists': 5}
--------------------------------------------------------------------------------
Called __getattribute__ __dict__
Before: {}
Called __getattribute__ foo
Not have this value
foo: Value for foo
Called __getattribute__ foo
foo: Value for foo
Called __getattribute__ __dict__
After: {'foo': 'Value for foo'}
--------------------------------------------------------------------------------
Before: {}
Called __setattr__ data5
After: {'data': 5}
Called __setattr__ data10
After: {'data': 10}


#  第33条：用元类来验证子类

In [4]:
class Meta(type):
    def __new__(meta, name, bases, class_dict):
        print(meta, name, bases, class_dict)
        return type.__new__(meta, name, bases, class_dict)
    
class MyClass(object, metaclass=Meta):
    stuff = 123
    
    def foo(self):
        pass
    
class MyClassPython2(object):
    __metaclass__ = Meta
    
    def foo(self):
        pass
        

<class '__main__.Meta'> MyClass (<class 'object'>,) {'foo': <function MyClass.foo at 0x0000000004B60E18>, 'stuff': 123, '__module__': '__main__', '__qualname__': 'MyClass'}


In [9]:
class ValidatePolygon(type):
    def __new__(meta, name, bases, class_dict):
        if bases != (object,):
            if class_dict['sides'] < 3:
                raise ValueError("Plygons need 3+ sides")
        return type.__new__(meta, name, bases, class_dict)

class Ploygon(object, metaclass=ValidatePolygon):
    sides = None
    
    @classmethod
    def interior_angles(cls):
        return (cls.sides - 2) * 180
    
class Triangle(Ploygon):
    sides = 3
print(Triangle.interior_angles())


# class Line(Ploygon):
#     print("Before sides")
#     sides = 1
#     print("After sides")
#     print("After class")
# ValueError: Plygons need 3+ sides



180


# 第34条：用元类来注册子类

In [16]:
import simplejson as json


registry = {}
def registry_class(target_class):
    registry[target_class.__name__] = target_class
    
def deserializable(data):
    params = json.loads(data)
    name = params['class']
    target_class = registry[name]
    return target_class(*params["args"])

class Meta(type):
    def __new__(meta, name, bases, class_dict):
        cls = type.__new__(meta, name, bases, class_dict)
        registry_class(cls)
        return cls

class Serializable(object):
    def __init__(self, *args):
        self.args = args

    def serializable(self):
        return json.dumps(
        {
            "class": self.__class__.__name__,
            "args": self.args
        })
    
class Registeredserializable(Serializable, metaclass=Meta):
    pass


class Vector3D(Registeredserializable):
    def __init__(self, x, y, z):
        super().__init__(x, y, z)
        self.x, self.y, self.z = x, y, z
        
v3 = Vector3D(10, -7, 3)
print("Before:  ", v3)
data = v3.serializable()
print("Serialized:  ", data)
print("After:  ", deserializable(data))

Before:   <__main__.Vector3D object at 0x0000000004C57FD0>
Serialized:   {"args": [10, -7, 3], "class": "Vector3D"}
After:   <__main__.Vector3D object at 0x0000000004C49320>


# 第40条：考虑用协程来并发地运行多个函数

In [1]:
def my_coroutine():
    while True:
        received = yield
        print("Received: ", received)
        
it = my_coroutine()
it.send(None)
it.send('First')
it.send('Second')


Received:  First
Received:  Second


In [7]:
def delegated():
    yield 1
    yield 2
    
def composed():
    yield 'A'
    # python2 usage
    for value in delegated():
        yield value
    yield 'B'
    
def another_composed():
    yield 'A'
    # python3 usage
    yield from delegated()
    yield 'B'

print(list(composed()))
print(list(another_composed()))
# print()

['A', 1, 2, 'B']
['A', 1, 2, 'B']


In [1]:
class MyReturn(Exception):
    def __init__(self, value):
        self._value = value
        
def delegated():
    yield 1
    raise MyReturn(2)
    yield 2
    
def composed():
    try:
        for value in delegated():
            yield value
    except MyReturn as e:
        output = e._value
    yield output*2 

print(list(composed()))

[1, 4]



# 第41条：考虑用concurrent.futures来实现真正的平行计算

In [ ]:
from time import time
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor
from multiprocessing import Process as pro, Pool as ProPool
from multiprocessing.dummy import Process as thr, Pool as ThrPool
from multiprocessing import freeze_support


def get_cp(pair):
    a, b = pair
    low = min(a, b)
    for i in range(low, 0, -1):
        if a % i == 0 and b % i == 0:
            return i

numbers = [(196330, 2265973), (2030677, 3814172),
           (1551645, 2229620), (2039045, 2020802)]

if __name__ == "__main__":
    freeze_support()

    start = time()
    result = list(map(get_cp, numbers))
    end = time()
    print("Cost time: {:.5f}".format(end - start))

    # process_list = []
    # start = time()
    # for number in numbers:
    #     p = pro(target=get_cp, args=number)
    #     p.start()
    #     process_list.append(p)
    # for process in process_list:
    #     process.join()
    # end = time()
    # print("Cost time: {:.5f}".format(end - start))

    # thread_list = []
    # start = time()
    # for number in numbers:
    #     p = pro(target=get_cp, args=number)
    #     p.start()
    #     thread_list.append(p)
    # for thread in thread_list:
    #     thread.join()
    # end = time()
    # print("Cost time: {:.5f}".format(end - start))

    start = time()
    pool = ProPool(4)
    result = list(pool.map(get_cp, numbers))
    pool.close()
    pool.join()
    end = time()
    print("Cost time: {:.5f}".format(end - start))

    start = time()
    pool = ThrPool(4)
    result = list(pool.map(get_cp, numbers))
    pool.close()
    pool.join()
    end = time()
    print("Cost time: {:.5f}".format(end - start))

    start = time()
    pool = ThreadPoolExecutor(max_workers=4)
    result = list(pool.map(get_cp, numbers))
    end = time()
    print("Cost time: {:.5f}".format(end - start))

    start = time()
    pool = ProcessPoolExecutor(max_workers=4)
    result = list(pool.map(get_cp, numbers))
    end = time()
    print("Cost time: {:.5f}".format(end - start))


In [1]:

numbers = [(196330, 2265973), (2030677, 3814172),
           (1551645, 2229620), (2039045, 2020802),
           (15523645, 22234920), (20393405, 2020232)]
for number in numbers:
    print(number)

(196330, 2265973)
(2030677, 3814172)
(1551645, 2229620)
(2039045, 2020802)
(15523645, 22234920)
(20393405, 2020232)


# 第47条：在重视精确度的场合，应该使用decimal

In [8]:
from decimal import Decimal, ROUND_UP, ROUND_DOWN
rate = Decimal('1.45')
seconds = Decimal(str(3*60+42))
cost = rate * seconds / Decimal('60')
print(cost)
rounded_up = cost.quantize(Decimal('0.01'), rounding=ROUND_UP)
rounded_down = cost.quantize(Decimal('0.01'), rounding=ROUND_DOWN)
# cost.quantize??
print(rounded_up)
print(rounded_down)

5.365
5.37
5.36


In [18]:
s = "abc123abc456"
print(s.find('abc', 4))

from operator import methodcaller
print(methodcaller('find','abc',4)(s))

6
6


In [30]:
class Circle:
    def __init__(self, r):
        self._r = r
        
    def get_area(self):
        return self._r **2*3.14

class Rectangle:
    def __init__(self, w, h):
        self._w = w
        self._h = h
        
    def get_area(self):
        return self._w * self._h
    
class Triangle:
    def __init__(self, a, b, c):
        self._a = a
        self._b = b
        self._c = c
    
    def get_area(self):
        a, b, c = self._a, self._b, self._c
        p = (a + b + c)/2
        area = p*(p-a)*(p-b)*(p-c)
        return area**0.5
    
def Get_Area(shape):
    f = getattr(shape, 'get_area', None)
    if f:
        return f()

t = Triangle(3, 4, 5)
c = Circle(5)
r = Rectangle(3, 4)
print(list(map(Get_Area, [t, c, r])))

for s in [t, c, r]:
    print(getattr(s, 'get_area')())
        

[6.0, 78.5, 12]
6.0
78.5
12


In [41]:
class Attr:
    def __init__(self, name, type_):
        self._name = name
        self._type = type_
    
    def __get__(self, instance, owner):
        return instance.__dict__[self._name]
    
    def __set__(self, instance, value):
        if not isinstance(value, self._type):
            raise TypeError("Type isn't except")
        instance.__dict__[self._name] = value
    
    def __delete__(self, instance):
        del instance.__dict__[self._name]
        
class Person:
    name = Attr('name', str)
    age = Attr('age', int)
    height = Attr('height', float)
p = Person()
p.name = "zxc"
print(p.name)

zxc


In [52]:
from functools import total_ordering
import abc

@total_ordering
class Shape(abc.ABC):
#     __metaclass__=abc.ABCMeta
    @abc.abstractmethod
    def get_area(self):
        pass
    
    def __lt__(self, other):
        if not isinstance(other, Shape):
            raise TypeError("excepted the instance is Shape")
        return self.get_area() < other.get_area()

    def __eq__(self, other):
        if not isinstance(other, Shape):
            raise TypeError("excepted the instance is Shape")
        return self.get_area() == other.get_area()

class Circle(Shape):
    def __init__(self, r):
        self._r = r
        
    def get_area(self):
        return self._r **2*3.14

class Rectangle(Shape):
    def __init__(self, w, h):
        self._w = w
        self._h = h
        
    def get_area(self):
        return self._w * self._h
    
class Triangle(Shape):
    def __init__(self, a, b, c):
        self._a = a
        self._b = b
        self._c = c
    
    def get_area(self):
        a, b, c = self._a, self._b, self._c
        p = (a + b + c)/2
        area = p*(p-a)*(p-b)*(p-c)
        return area**0.5
    
t = Triangle(3, 4, 5)
c = Circle(5)
print(t > c)
print(t < c)
print(t == c)


False
True
False


In [59]:
class IntTuple(tuple):
    def __new__(cls, iterable):
        g = (i for i in iterable if isinstance(i, int) and i >0)
#         return super().__new__(cls, iterable)
#         return super().__new__(cls, g)
        return super(IntTuple, cls).__new__(cls, g)
    def __init__(self, iterable):
#         super().__init__(iterable)
        super(IntTuple, self).__init__()
    
it = IntTuple([1, -1, 'abc', 6, ['x', 'y'], 3])
print(it)

(1, 6, 3)
